In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import glob
from tqdm import tqdm


def load_aircraft_turbulence_data(data_path, flight_date='2022-04-01', verbose=True):
    """
    Load high-frequency aircraft turbulence data from ASC files.
    
    Parameters:
    -----------
    data_path : str
        Path to the ASC data file containing nose-boom measurements
    flight_date : str
        Date string for proper time indexing
    
    Returns:
    --------
    xr.Dataset
        Xarray dataset with time-indexed atmospheric variables
    """
    if verbose:
        print(f"📂 Loading aircraft data from: {data_path}")

    # read the first line to determine the column names
    with open(data_path, 'r') as f:
        header = f.readline().strip()

    cols = header.split()[1:]  # Skip the first column which is just "!"
    if verbose:
        print(f"📝 Columns: {cols}")
    # Load ASCII data file (whitespace separated)
    df = pd.read_csv(data_path, sep=r'\s+', index_col=False, names=cols, skiprows=1,)

    # coerce numeric columns to float, errors='coerce' will turn non-convertible values into NaN
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    if verbose:
        print(f"✅ Raw data loaded: {len(df)} records")

    # Create proper time coordinate from UTC seconds
    # UTC column contains seconds since start of day
    start_time = pd.to_datetime(flight_date) + pd.to_timedelta(df['UTC'].min(), unit='s')
    end_time = pd.to_datetime(flight_date) + pd.to_timedelta(df['UTC'].max(), unit='s')
    
    # Generate time index with even spacing
    df['time'] = pd.date_range(start=start_time, end=end_time, periods=len(df))
    df = df.set_index('time')
    
    # Convert to xarray for better handling of multi-dimensional data
    ds = xr.Dataset.from_dataframe(df)
    
    if verbose:
        print(f"📊 Dataset created with {len(ds.time)} time points")
        print(f"🕒 Time range: {ds.time.min().values} to {ds.time.max().values}")

        # Calculate and display sampling characteristics
        dt = np.median(np.diff(ds.time.values)) / np.timedelta64(1, 's')
        sampling_rate = 1.0 / dt
        print(f"📈 Sampling rate: {sampling_rate:.1f} Hz")
        print(f"⏱️ Time resolution: {dt:.3f} seconds")

    return ds


tb_files = glob.glob('../../../data/observations/raw/t-bird/noseboom/100hz/Flight_*_P6_*_100Hz_tb.asc')
p6_files = glob.glob('../../../data/observations/raw/polar6/noseboom/100hz/Flight_*_P6_*_100Hz.asc')

ds_tb_list = []
ds_p6_list = []

for tb_file in tqdm(tb_files):
    date = tb_file.split('_')[-5]
    date = pd.to_datetime(date).strftime('%Y-%m-%d')
    ds_tb = load_aircraft_turbulence_data(data_path=tb_file, flight_date=date, verbose=False)
    ds_tb_list.append(ds_tb)
    # save to netcdf

for p6_file in tqdm(p6_files):
    date = p6_file.split('_')[-4]
    date = pd.to_datetime(date).strftime('%Y-%m-%d')
    ds_p6 = load_aircraft_turbulence_data(data_path=p6_file, flight_date=date, verbose=False)
    ds_p6_list.append(ds_p6)

ds_tb = xr.concat(ds_tb_list, dim='time').sortby('time')
ds_p6 = xr.concat(ds_p6_list, dim='time').sortby('time')

ds_p6.to_netcdf('../../../data/observations/processed/polar6_unified/noseboom_unified.nc')
ds_tb.to_netcdf('../../../data/observations/processed/t-bird_unified/noseboom_unified.nc')


 50%|█████     | 3/6 [00:03<00:03,  1.12s/it]/projekt_agmwend/home_rad/Joshua/tmp/ipykernel_684187/727575787.py:35: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path, sep=r'\s+', index_col=False, names=cols, skiprows=1,)
100%|██████████| 13/13 [00:21<00:00,  1.63s/it]
